In [1]:
import nest_asyncio
nest_asyncio.apply()

In [2]:
import collections

import numpy as np
import tensorflow as tf
import os
import tensorflow_federated as tff
import functools
import time

np.random.seed(0)

tff.federated_computation(lambda: 'Check-OK')()

b'Check-OK'

## Load pre-trained model

### Generate the vocabulary lookup tables

In [3]:
vocab = list('dhlptx@DHLPTX $(,048cgkoswCGKOSW[_#\'/37;?bfjnrvzBFJNRVZ"&*.26:\naeimquyAEIMQUY]!%)-159\r')

char2idx = {u:1 for i, u in enumerate(vocab)}
idx2char = np.array(vocab)


### Load pre-trained model and generate random text

In [4]:
def load_model(batch_size):
    urls = {1: 'https://storage.googleapis.com/tff-models-public/dickens_rnn.batch1.kerasmodel', 
            8: 'https://storage.googleapis.com/tff-models-public/dickens_rnn.batch8.kerasmodel'}
    assert batch_size in urls, 'batch_size must be in ' + str(urls.keys())
    url = urls[batch_size]
    local_file = tf.keras.utils.get_file(os.path.basename(url), origin=url)
    return tf.keras.models.load_model(local_file, compile=False)

In [5]:
def generate_text(model, start_string):
    # from tensorflow.org/tutorials/sequences/text_generation
    num_generate = 200
    input_eval = [char2idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)
    text_generated = []
    temperature = 1.0
    
    model.reset_states()
    for i in range(num_generate):
        predictions = model(input_eval)
        predictions = tf.squeeze(predictions, 0)
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(
            predictions, num_samples=1)[-1,0].numpy()
        input_eval = tf.expand_dims([predicted_id],0)
        text_generated.append(idx2char[predicted_id])
        
    return (start_string + ''.join(text_generated))

In [7]:
keras_model_batch1 = load_model(batch_size=1)
print(generate_text(keras_model_batch1, 'Where art thou tensorflow, you ask? '))

Where art thou tensorflow, you ask? ere is he?"

In any name.

He was suddenly seee of scarecrows or MissoPulate! Which is he, by money;
and far dear father, and it was histoors contended with Notre Manette, received him to the dri


## Load Federated dataset and pre-process

In [8]:
train_data, test_data = tff.simulation.datasets.shakespeare.load_data()

In [9]:
#example look at data
raw_example_dataset = train_data.create_tf_dataset_for_client(
    'THE_TRAGEDY_OF_KING_LEAR_KING')
for x in raw_example_dataset.take(2):
    print(x['snippets'])

tf.Tensor(b'', shape=(), dtype=string)
tf.Tensor(b'What?', shape=(), dtype=string)


In [10]:
# Input pre-processing parameters
SEQ_LENGTH = 100
BATCH_SIZE = 8
BUFFER_SIZE = 100  # For dataset shuffling

In [11]:
# Construct a lookup table to map string chars to indexes,
# using the vocab loaded above:
table = tf.lookup.StaticHashTable(
    tf.lookup.KeyValueTensorInitializer(
        keys=vocab, values=tf.constant(list(range(len(vocab))),dtype=tf.int64)),default_value=0)

def to_ids(x):
  s = tf.reshape(x['snippets'], shape=[1])
  chars = tf.strings.bytes_split(s).values
  ids = table.lookup(chars)
  return ids

def split_input_target(chunk):
  input_text = tf.map_fn(lambda x: x[:-1], chunk)
  target_text = tf.map_fn(lambda x: x[1:], chunk)
  return (input_text, target_text)

def preprocess(dataset):
  return (
      # Map ASCII chars to int64 indexes using the vocab
      dataset.map(to_ids)
      # Split into individual chars
      .unbatch()
      # Form example sequences of SEQ_LENGTH +1
      .batch(SEQ_LENGTH + 1, drop_remainder=True)
      # Shuffle and form minibatches
      .shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)
      # And finally split into (input, target) tuples,
      # each of length SEQ_LENGTH.
      .map(split_input_target))

In [12]:
example_dataset = preprocess(raw_example_dataset)
print(example_dataset.element_spec)

(TensorSpec(shape=(8, 100), dtype=tf.int64, name=None), TensorSpec(shape=(8, 100), dtype=tf.int64, name=None))


## Compile model and test pre-processed data

In [13]:
class FlattenedCategoricalAccuracy(tf.keras.metrics.SparseCategoricalAccuracy):

  def __init__(self, name='accuracy', dtype=tf.float32):
    super().__init__(name, dtype=dtype)

  def update_state(self, y_true, y_pred, sample_weight=None):
    y_true = tf.reshape(y_true, [-1, 1])
    y_pred = tf.reshape(y_pred, [-1, len(vocab), 1])
    return super().update_state(y_true, y_pred, sample_weight)

In [14]:
BATCH_SIZE = 8  # The training and eval batch size for the rest of this tutorial.
keras_model = load_model(batch_size=BATCH_SIZE)
keras_model.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[FlattenedCategoricalAccuracy()])

# Confirm that loss is much lower on Shakespeare than on random data

loss, accuracy = keras_model.evaluate(example_dataset.take(5), verbose=0)
print('Evaluating on an example Shakespeare character: {a:3f}'.format(a=accuracy))

# As a sanity check, can construct some completely random data, where expect
# the accuracy to be essentially random:

random_guessed_accuracy = 1.0 / len(vocab)
print('Expected accuracy for random guessing: {a:.3f}'.format(a=random_guessed_accuracy))

random_indexes = np.random.randint(low=0, high=len(vocab), size=1 * BATCH_SIZE * (SEQ_LENGTH + 1))
data = collections.OrderedDict(snippets=tf.constant(''.join(np.array(vocab)[random_indexes]), shape=[1, 1]))
random_dataset = preprocess(tf.data.Dataset.from_tensor_slices(data))
loss, accuracy = keras_model.evaluate(random_dataset, steps=10, verbose=0)
print('Evaluating on completely random data: {a:.3f}'.format(a=accuracy))

Evaluating on an example Shakespeare character: 0.398000
Expected accuracy for random guessing: 0.012


Evaluating on completely random data: 0.011


## Fine-tune model with FL

In [15]:
# Clone the keras_model inside `create_tff_model()`, which TFF will
# call to produce a new copy of the model inside the graph that it will 
# serialize. Note: want to construct all the necessary objects needed
# _inside_ this method.

def create_tff_model():
  # TFF uses an `input_spec` so it knows the types and shapes
  # that model expects.
  input_spec = example_dataset.element_spec
  keras_model_clone = tf.keras.models.clone_model(keras_model)
  return tff.learning.from_keras_model(
      keras_model_clone,
      input_spec=input_spec,
      loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
      metrics=[FlattenedCategoricalAccuracy()])

In [16]:
# This command builds all the TensorFlow graphs and serializes them: 
fed_avg = tff.learning.build_federated_averaging_process(
    model_fn=create_tff_model,client_optimizer_fn=lambda: tf.keras.optimizers.SGD(learning_rate=0.5))

In [17]:
state = fed_avg.initialize()
state, metrics = fed_avg.next(state, [example_dataset.take(5)])
train_metrics = metrics['train']
print('loss={l:.3f}, accuracy={a:.3f}'.format(l=train_metrics['loss'], a=train_metrics['accuracy']))

loss=4.399, accuracy=0.142


In [18]:
def data(client, source=train_data):
  return preprocess(source.create_tf_dataset_for_client(client)).take(5)

clients = ['ALL_S_WELL_THAT_ENDS_WELL_CELIA', 'MUCH_ADO_ABOUT_NOTHING_OTHELLO',]

train_datasets = [data(client) for client in clients]

# concatenate the test datasets for evaluation with Keras by creating a 
# Dataset of Datasets, and then identity flat mapping across all the examples.

test_dataset = tf.data.Dataset.from_tensor_slices([data(client, test_data) for client in clients]).flat_map(lambda x: x)

In [19]:
NUM_ROUNDS = 5

# The state of the FL server, containing the model and optimization state.
state = fed_avg.initialize()

# Load pre-trained Keras model weights into the global model state.
state = tff.learning.state_with_new_model_weights(
    state,
    trainable_weights=[v.numpy() for v in keras_model.trainable_weights],
    non_trainable_weights=[v.numpy() for v in keras_model.non_trainable_weights])


def keras_evaluate(state, round_num):
  # Take  global model weights and push them back into a Keras model to
  # use its standard `.evaluate()` method.
  keras_model = load_model(batch_size=BATCH_SIZE)
  keras_model.compile(
      loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
      metrics=[FlattenedCategoricalAccuracy()])
  state.model.assign_weights_to(keras_model)
  loss, accuracy = keras_model.evaluate(example_dataset, steps=2, verbose=0)
  print('\tEval: loss={l:.3f}, accuracy={a:.3f}'.format(l=loss, a=accuracy))


for round_num in range(NUM_ROUNDS):
  print('Round {r}'.format(r=round_num))
  keras_evaluate(state, round_num)
  state, metrics = fed_avg.next(state, train_datasets)
  train_metrics = metrics['train']
  print('\tTrain: loss={l:.3f}, accuracy={a:.3f}'.format(
      l=train_metrics['loss'], a=train_metrics['accuracy']))

print('Final evaluation')
keras_evaluate(state, NUM_ROUNDS + 1)

Round 0
	Eval: loss=3.304, accuracy=0.416
	Train: loss=4.304, accuracy=0.094
Round 1
	Eval: loss=4.255, accuracy=0.162
	Train: loss=4.134, accuracy=0.232
Round 2
	Eval: loss=4.115, accuracy=0.168
	Train: loss=3.987, accuracy=0.242
Round 3


	Eval: loss=4.024, accuracy=0.168
	Train: loss=3.873, accuracy=0.228
Round 4


	Eval: loss=3.938, accuracy=0.157
	Train: loss=3.784, accuracy=0.220
Final evaluation
	Eval: loss=3.830, accuracy=0.177


In [21]:
# Set newly trained weights back in the originally created model.
keras_model_batch1.set_weights([v.numpy() for v in keras_model.weights])
# Text generation requires batch_size=1
print(generate_text(keras_model_batch1, 'Where art thou tensorflow, you ask? '))

Where art thou tensorflow, you ask? en such profligates?"

"Is it will do," said Monsieurshed, in a mother and the heaves
was put in opinion, until I should have still secrete out of their dismissal from
the side of the day. But, no
